### Recitation 9 Trees

April 2022

Yisong Wang

# Self-recursive Data Structures

In [12]:
class SimpleSinglyLinkedList:  # Node
  def __init__(self, element, nextNode = None):
    # self is a local variable. The type of self is created from the very start
    # Re-assigning self is dubious throughout the class writing
    # At the very least, changing the class of self is forbidden
    self._element = element
    self._next = nextNode
    # the philosophy of this class, is that every node is the head 
    # and the head identifies the Singly Linked List
    # Therefore, it is against the phil that we have an empty list
    # i.e., we create a node which is a None
  
  def __len__(self):
    if self._next == None:
      return 1
    return 1 + len(self._next)

  def insert_from_head(self, e):
    prevNode = SimpleSinglyLinkedList(self._element, self._next)
    self.__init__(e, prevNode)
    # self.__init__(e, self)
  
  def insert_from_tail(self, e):
    while self._next is not None:
      self = self._next
    self._next = SimpleSinglyLinkedList(e, None)
  
  def delete_from_head(self):
    if self._next == None:
      raise TypeError('cannot delete till Empty')
    self.__init__(self._next._element, self._next._next)
  
  def __str__(self):
    result = []
    # curNode = self
    while self is not None:
        result.append(str(self._element) + "-->")
        self = self._next
    result.append("None")
    return "".join(result)

In [11]:
a = SimpleSinglyLinkedList(9, SimpleSinglyLinkedList(1, SimpleSinglyLinkedList(10, None)))

print(len(a))
print(a)

a.insert_from_head(100)
print(a._element)
print(a._next._element)
print(a._next._next._element)

3
9-->1-->10-->None
100
100
100


In [4]:
a = SimpleSinglyLinkedList(1) # this is really a node
# simple data structrues, we refer to Nodes as SLL
print(a)
print('len1', len(a))

a.insert_from_head(9)
print(a)
a.insert_from_tail(10)
print(a)

print('len2', len(a))

a.delete_from_head()
print(a)


1-->None
len1 1
9-->1-->None
9-->1-->10-->None
len2 3
1-->10-->None


# Binary Trees

In [16]:
class Empty(Exception):
    pass


# Exercise 1 Implement a tree class: with the idea of recursion s.t. tree is the same as a root
class TreeWithoutParent:  # This is a tree, at the same time a node
    def __init__(self, element=None, left=None, right=None):
        self._element = element
        self._left = left
        self._right = right
        # self._size = 0
        # if left is not None:
        #     self._size += left._size
        # if right is not None:
        #     self._size += right._size

    # boolean checks:
    def is_leaf(self):
        return self._left is None and self._right is None

    # basic properties:
    def __len__(self):   # how many nodes in this tree, O(n)
        if self._element is None:
            return 0
        if self.is_leaf():
            return 1
        if self._left is None:
            return len(self._right) + 1
        if self._right is None:
            return len(self._left) + 1
        return len(self._left) + len(self._right) + 1


    def height(self):   # calculating height is an O(n) operation
        if self.is_leaf():
            return 0
        left_height = self._left.height() if self._left is not None else -1
        right_height = self._right.height() if self._right is not None else -1
        return max(left_height, right_height) + 1 # counting from 0

    def depth(self, e, curdepth=0):  # the first appearance of e in preOrder search
        if self._element == e:
            return curdepth
        if self.is_leaf():  # my tree contains one node and onde node only
            return None  # e is nonexistence in my tree self
        if self._left is not None:
            search_left = self._left.depth(e, curdepth + 1)
            if search_left is not None:
                return search_left
        return self._right.depth(e, curdepth + 1)

    # traversals: one of its maneuvoured application in calculating depth
    def preOrderTraverse(self, curdepth=0):
        yield self._element, curdepth
        
        if self._left is not None:
            for each, dep in self._left.preOrderTraverse(curdepth + 1):
              yield each, dep
        
        if self._right is not None:
            for each, dep in self._right.preOrderTraverse(curdepth + 1):
              yield each, dep
        # return

    def postOrderTraverse(self, curdepth=0):
        if self._left is not None:
            yield from self._left.postOrderTraverse(curdepth + 1)
        if self._right is not None:
            yield from self._right.postOrderTraverse(curdepth + 1)
        yield self._element, curdepth

    def inOrderTraverse(self, curdepth = 0):
        if self._left is not None:
            yield from self._left.inOrderTraverse(curdepth + 1)
        yield self._element, curdepth
        if self._right is not None:
            yield from self._right.inOrderTraverse(curdepth + 1)

    def bfTraverse(self, this_level_node = None, curdepth =0):  # Level order traverse
        if this_level_node is None:
            this_level_node = (self,)
        if len(this_level_node) == 0:  # () == None is false
            return
        next_level_node = ()
        for node in this_level_node:
            if node._left is not None:
                next_level_node += (node._left,)
            
            if node._right is not None:
                next_level_node += (node._right,)

            yield node._element, curdepth
            
        yield from self.bfTraverse(next_level_node, curdepth + 1)

        # if self._left is None:
        #     if self._right is None:
        #         return None
        #     return self._right.depth(e,curdepth+1)
        # search_left = self._left.depth(e, curdepth + 1 )
        # return search_left if search_left is not None else self._right.depth(e, curdepth+1)

    def search_in_tree(self, e):
        for each, depth in self.bfTraverse():
            if each == e:
                return each, depth
        return None, None

    # access through binary complete arguments, which by convention starts from 1
    def __getitem__(self, item):  # parent is //2
        if item == 1:
            return self
        parent = self[item // 2]
        if item % 2 == 0:
            if parent._left is None:
                raise Empty("the required node index is flawed from: ", item//2)
            return parent._left
        if item % 2 == 1:
            if parent._right is None:
                raise Empty("the required node index is flawed from: ", item//2)
            return parent._right

    def __setitem__(self, key, value):
        if key == 1:
            self._element = value
            return
        parent = self[key//2]
        if key % 2 == 0:
            if parent._left is None:
                parent._left = TreeWithoutParent(value)
            else:
                parent._left._element = value
            # return parent._left
        if key % 2 == 1:
            if parent._right is None:
                parent._right = TreeWithoutParent(value)
            else:
                parent._right._element = value
            # return parent._right

    def generate_from_complete_list(self, complete_list):
        for idx, each in enumerate(complete_list):
            if each != '#':
                self[idx+1]=each

    # tree representations
    def __str__(self):
        return str(self._element)

    def pretty_print(self):

        def all_elements_are_None(list_of_nodes):
            for each in list_of_nodes:
                if each is not None:
                    return False
            return True

        def print_spaces(number):
            for i in range(number):
                print("  ", end="")

        def print_internal(this_level_nodes, current_level, max_level):
            if (len(this_level_nodes) == 0 or all_elements_are_None(this_level_nodes)): # 前一种手段可以直接控制，是None就避免append
                return  # Base case of recursion: out of nodes, or only None left
            floor = max_level - current_level;
            endgeLines = 2 ** max(floor - 1, 0);  # 2^1=2
            firstSpaces = 2 ** floor - 1;  # 2^2-1 = 3
            betweenSpaces = 2 ** (floor + 1) - 1;  # 2^3-1= 7
            print_spaces(firstSpaces)
            next_level_nodes = []
            for node in this_level_nodes:
                if (node is not None):
                    print(node._element, end="")  # i.e.避免换行的print
                    next_level_nodes.append(node._left)
                    next_level_nodes.append(node._right)
                else:
                    next_level_nodes.append(None)
                    next_level_nodes.append(None)
                    print_spaces(1)
                print_spaces(betweenSpaces)
            print()  # 换行
            for i in range(1, endgeLines + 1):  # 这个目的是为了加上list长度，然后结果更好看，约定层的edge越大
                for j in range(0, len(this_level_nodes)):
                    print_spaces(firstSpaces - i)
                    if (this_level_nodes[j] == None):
                        print_spaces(endgeLines + endgeLines + i + 1);
                        continue
                    if (this_level_nodes[j]._left != None):
                        print("/", end="")
                    else:
                        print_spaces(1)
                    print_spaces(i + i - 1)
                    if (this_level_nodes[j]._right != None):
                        print("\\", end="")
                    else:
                        print_spaces(1)
                    print_spaces(endgeLines + endgeLines - i)
                print()
            print_internal(next_level_nodes, current_level + 1, max_level)

        levels = self.height()+1  # Need a function to calculate levels. Use 3 for now.
        print_internal([self], 1, levels)


In [17]:
tree = TreeWithoutParent('+', TreeWithoutParent('*', TreeWithoutParent('3'),TreeWithoutParent('2')),TreeWithoutParent('-', TreeWithoutParent('5', TreeWithoutParent('30'), None), TreeWithoutParent('2')) )
tree.pretty_print()

print(tree.height())
print(tree._right._left._left.height())
print(tree.depth('5'))
print(tree.depth('30'))
print('size:',len(tree))


height1 = tree.height()
print('tree height', height1)

print("---------------------predorder------------------------")
for each, depth in tree.preOrderTraverse():
  print(each, depth)

print("---------------------indorder------------------------")
for each, depth in tree.inOrderTraverse():
  print(each, depth)

print("---------------------postdorder------------------------")
for each, depth in tree.postOrderTraverse():
  print(each, depth)

print("---------------------breadth-frist-Traverse------------------------")
for each, depth in tree.bfTraverse():
  print(each, depth)



              +                              
            /  \              
          /      \            
        /          \          
      /              \        
      *              -              
    /  \          /  \      
  /      \      /      \    
  3      2      5      2      
                /              
                30              
                                                                
3
0
2
3
size: 8
tree height 3
---------------------predorder------------------------
+ 0
* 1
3 2
2 2
- 1
5 2
30 3
2 2
---------------------indorder------------------------
3 2
* 1
2 2
+ 0
30 3
5 2
- 1
2 2
---------------------postdorder------------------------
3 2
2 2
* 1
30 3
5 2
2 2
- 1
+ 0
---------------------breadth-frist-Traverse------------------------
+ 0
* 1
- 1
3 2
2 2
5 2
2 2
30 3


In [ ]:
for idx in range(1,2**(height1+1)):
  try:
      print('access according to binary complete: ', tree[idx])
  except Empty as e:
      print('error', e)

tree[8]='100'
print(tree[8])
for i in tree.preOrderTraverse():
  print(i, end="")
print()

for i in tree.postOrderTraverse():
  print(i, end="")
print()

for i in tree.inOrderTraverse():
  print(i, end="")
print()

for i in tree.bfTraverse():
  print(i,end="")
print()

print(tree.search_in_tree('30'))

In [ ]:
tree1 = TreeWithoutParent()
print(tree1)
print(len(tree1), tree1.height())
tree1.generate_from_complete_list([10,5,12,1,7,'#',15,'#',2,6,'#','#','#',14,'#'])
for i in tree1.preOrderTraverse():
    print(i, end="")
for i in tree1.postOrderTraverse():
    print(i, end="")
print()
for i in tree1.inOrderTraverse():
    print(i, end="")
print()
for i in tree1.bfTraverse():
    print(i, end="")
print()
print(len(tree1), tree1.height())
print(tree1)
tree1.pretty_print()

None
0 0
(10, 0)(5, 1)(1, 2)(2, 3)(7, 2)(6, 3)(12, 1)(15, 2)(14, 3)(2, 3)(1, 2)(6, 3)(7, 2)(5, 1)(14, 3)(15, 2)(12, 1)(10, 0)
(1, 2)(2, 3)(5, 1)(6, 3)(7, 2)(10, 0)(12, 1)(14, 3)(15, 2)
(10, 0)(5, 1)(12, 1)(1, 2)(7, 2)(15, 2)(2, 3)(6, 3)(14, 3)
9 3
10
              10                              
            /  \              
          /      \            
        /          \          
      /              \        
      5              12              
    /  \              \      
  /      \              \    
  1      7              15      
    \  /              /      
    2  6              14      
                                                                
